# Measurement of $R_{500}$ through iterative gas-density profile generation

## Import Statements

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from astropy.units import Quantity
from astropy.cosmology import LambdaCDM
from shutil import rmtree, copyfile
import os
from matplotlib import pyplot as plt
from tqdm import tqdm

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 50
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.tools import gas_mass_radius_pipeline
# from xga.relations.fit import scaling_relation_odr, scaling_relation_curve_fit
# from xga.models.misc import straight_line, power_law
from xga.sources import NullSource
from xga.sas.phot import evselect_image

%matplotlib inline

## Defining the cosmology

We setup the LoVoCCS-chosen cosmology:

In [2]:
cosmo = LambdaCDM(71, 0.2648, 0.7352, Ob0=0.0448)

## Defining useful values

Here we define any widely used constants:

In [3]:
# The energy bands we wish to measure luminosity within
# lum_en = Quantity([[0.5, 2.0], [0.01, 100.0], [0.1, 2.4]], 'keV')

## Reading in the sample

We need the information from our sample file to declare the XGA ClusterSample further down, primarily the location and redshift, but we also make use of the MCXC estimate of $R_{500}$ to get some idea of the scale of these objects, even if we don't fully trust that measurement:

In [4]:
samp = pd.read_csv('../../sample_files/X-LoVoCCSI.csv')
samp['LoVoCCS_name'] = samp['LoVoCCSID'].apply(lambda x: 'LoVoCCS-' + str(x))
og_samp = samp.copy()
samp

,LoVoCCSID,Name,start_ra,start_dec,MCXC_Redshift,MCXC_R500,MCXC_RA,MCXC_DEC,manual_xray_ra,manual_xray_dec,MCXC_Lx500_0.1_2.4,LoVoCCS_name
0,1,A2029,227.734300,5.745471,0.0766,1.3344,227.73000,5.720000,227.734300,5.745471,8.726709e+44,LoVoCCS-1
1,2,A401,44.740000,13.580000,0.0739,1.2421,44.74000,13.580000,NaN,NaN,6.088643e+44,LoVoCCS-2
2,4A,A85North,10.458750,-9.301944,0.0555,1.2103,10.45875,-9.301944,NaN,NaN,5.100085e+44,LoVoCCS-4A
3,4B,A85South,10.451487,-9.460007,0.0555,1.2103,10.45875,-9.301944,10.451487,-9.460007,5.100085e+44,LoVoCCS-4B
4,5,A3667,303.157313,-56.845978,0.0556,1.1990,303.13000,-56.830000,303.157313,-56.845978,4.871933e+44,LoVoCCS-5
...,...,...,...,...,...,...,...,...,...,...,...,...
62,121,A3128,52.466189,-52.580728,0.0624,0.8831,52.50000,-52.600000,52.466189,-52.580728,1.101682e+44,LoVoCCS-121
63,122,A1023,157.000000,-6.800000,0.1176,0.8553,157.00000,-6.800000,NaN,NaN,1.095941e+44,LoVoCCS-122
64,123,A3528,193.670000,-29.220000,0.0544,0.8855,193.67000,-29.220000,NaN,NaN,1.093054e+44,LoVoCCS-123
65,131,A761,137.651250,-10.581111,0.0916,0.8627,137.65125,-10.581111,NaN,NaN,1.063423e+44,LoVoCCS-131


Here we remove some of the existing columns from the sample, that way they won't be included in the output results file for the LTR run, as well as making sure the column names are those expected by the pipeline:

In [5]:
samp = samp[['LoVoCCS_name', 'start_ra', 'start_dec', 'MCXC_Redshift']]
samp = samp.rename(columns={'LoVoCCS_name': 'name', 'start_ra': 'ra', 'start_dec': 'dec', 
                            'MCXC_Redshift': 'redshift'})

## Running the XGA $M_{\rm{gas}}$-$R_{500}$ pipeline

In [6]:
# n_src = NullSource()
# n_src.info()

In [7]:
# evselect_image(n_src)

In [8]:
# convo = {'pn': 'PN', 'mos1': 'M1', 'mos2': 'M2'}

In [9]:
# with tqdm(desc="Fixing DAXA's fuck ups", total=len(n_src.obs_ids)) as onwards:
#     for oi in n_src.obs_ids:
#         xga_pth = "../xga_output/{oi}/".format(oi=oi)
#         ims = [f for f in os.listdir(xga_pth) if 'keVimg' in f and '0.5' in f]
#         for im in ims:
#             rel_im_pth = os.path.abspath(os.path.join(xga_pth, im))
            
#             # This is a real bodge but I really need the DAXA 
#             dest_pth = "../../../X-LoVoCCS-Data/data/archives/LoVoCCS/processed_data/xmm_pointed/{oi}/images/".format(oi=oi)
#             cur_inst = convo[im.split('_')[1]]
#             dest_f = "obsid{oi}-inst{i}-subexpALL-en0.5_2.0keV-image.fits".format(oi=oi, i=cur_inst)
#             dest_pth = os.path.abspath(os.path.join(dest_pth, dest_f))
#             copyfile(rel_im_pth, dest_pth)
#         onwards.update(1)

In [10]:
res_file = '../../outputs/results/gmr_r500_pipeline_results.csv'
rhist_file = '../../outputs/results/gmr_r500_radii_history.csv'

# clean_obs_threshold=0.9
srcs, results, rad_hist = gas_mass_radius_pipeline(samp, 500, 0.125, 'beta', 'king', Quantity(500, 'kpc'),timeout=Quantity(4, 'hr'), 
                                                   save_samp_results_path=res_file, save_rad_history_path=rhist_file, 
                                                   use_peak=False, cosmo=cosmo)

Declaring BaseSource Sample:  31%|███▏      | 21/67 [03:22<07:22,  9.62s/it]


TypeError: 'Image' object is not subscriptable

In [ ]:
stop

## Comparing results with MCXC values

We create some figures comparing the measured values to those in the MCXC catalogue that LoVoCCS was initially selected from. We load in the results files here (though we also output them to variables when running the pipelines, this allows the figure section of this notebook to be run by itself, without re-running the pipelines unnecessarily):

In [ ]:
results_mfree = pd.read_csv('../../outputs/results/ltr_r500_metfree_pipeline_results.csv')
results_mfree = results_mfree.rename(columns={'name': 'LoVoCCS_name'})
results = pd.read_csv('../../outputs/results/ltr_r500_pipeline_results.csv')
results = results.rename(columns={'name': 'LoVoCCS_name'})

results_mfree = pd.merge(results_mfree, og_samp, on='LoVoCCS_name')
results = pd.merge(results, og_samp, on='LoVoCCS_name')
results.tail(5)

In [ ]:
results_mfree.tail(5)

### Comparing MCXC and X-LoVoCCS luminosities

We measured luminosities in the 0.1-2.4 keV band that was utilised for MCXC, so we compare our measurements (with our own derived values of $R_{500}$) with the original measurements - this is interesting primarily because the MCXC luminosities were one of the key selection criteria for the LoVoCCS sample. Remember also that we ran with metallicity free and with it fixed, so two comparisons are necessary:

In [ ]:
lx_comp = scaling_relation_curve_fit(straight_line, Quantity(results['Lx500_0.1-2.4'], 'erg/s'), 
                                     Quantity(results[['Lx500_0.1-2.4-', 'Lx500_0.1-2.4+']], 'erg/s'), 
                                     Quantity(results['MCXC_Lx500_0.1_2.4'], 'erg/s'), 
                                     y_norm=Quantity(1e+44, 'erg/s'), 
                                     x_norm=Quantity(1e+44, 'erg/s'), point_names=results['LoVoCCS_name'].values, 
                                     y_name=r'$L^{0.1-2.4\rm{keV}}_{\rm{XMM},500}$', 
                                     x_name=r'$L^{0.1-2.4\rm{keV}}_{\rm{MCXC},500}$', 
                                     third_dim_info=results['redshift'].values, third_dim_name='Redshift')

lx_comp_mfree = scaling_relation_curve_fit(straight_line, Quantity(results_mfree['Lx500_0.1-2.4'], 'erg/s'), 
                                           Quantity(results_mfree[['Lx500_0.1-2.4-', 'Lx500_0.1-2.4+']], 'erg/s'), 
                                           Quantity(results_mfree['MCXC_Lx500_0.1_2.4'], 'erg/s'), 
                                           y_norm=Quantity(1e+44, 'erg/s'), 
                                           x_norm=Quantity(1e+44, 'erg/s'), point_names=results_mfree['LoVoCCS_name'].values, 
                                           y_name=r'$L^{0.1-2.4\rm{keV}}_{\rm{XMM},500}$', 
                                           x_name=r'$L^{0.1-2.4\rm{keV}}_{\rm{MCXC},500}$', 
                                           third_dim_info=results_mfree['redshift'].values, third_dim_name='Redshift')

#### Metallicity free

In [ ]:
lx_comp_mfree.view(figsize=(8, 7), label_points=False, log_scale=False, one_to_one=True, 
                   save_path='../../outputs/figures/global_properties/xmm_mcxc_lx_metfree_comp.pdf')
print(lx_comp_mfree.par_names)
lx_comp_mfree.pars

In [ ]:
lx_comp_mfree.view(figsize=(8, 7), label_points=True, log_scale=False, one_to_one=True)
{ind: lx_comp_mfree.point_names[ind] for ind in range(0, len(lx_comp_mfree.point_names))}

We also calculate and plot the distribution of ratios between the 0.1-2.4 keV luminosities, as another way of examining the same comparison:

In [ ]:
lx_rat = lx_comp_mfree.y_data[:, 0] / lx_comp_mfree.x_data[:, 0]

bins = np.arange(lx_rat.min(), lx_rat.max(), 0.05)

plt.figure(figsize=(6, 6))
plt.hist(lx_rat, bins=bins, color='firebrick', alpha=0.7)
plt.minorticks_on()
plt.tick_params(which='both', direction='in', top=True, right=True)
plt.axvline(1, linestyle='dashed', color='black')
plt.xlabel(r'$L^{0.1-2.4\rm{keV}}_{\rm{XMM},500} / L^{0.1-2.4\rm{keV}}_{\rm{MCXC},500}$', fontsize=15)
plt.ylabel('N', fontsize=15)
plt.tight_layout()
plt.savefig('../../outputs/figures/global_properties/xmm_mcxc_lx_metfree_ratio.pdf')
plt.show()

#### Metallicity fixed

In [ ]:
lx_comp.view(figsize=(8, 7), label_points=False, log_scale=False, one_to_one=True,
             save_path='../../outputs/figures/global_properties/xmm_mcxc_lx_comp.pdf')
print(lx_comp.par_names)
lx_comp.pars

In [ ]:
lx_comp.view(figsize=(8, 7), label_points=True, log_scale=False, one_to_one=True)
{ind: lx_comp.point_names[ind] for ind in range(0, len(lx_comp.point_names))}

We also calculate and plot the distribution of ratios between the 0.1-2.4 keV luminosities, as another way of examining the same comparison:

In [ ]:
lx_rat = lx_comp.y_data[:, 0] / lx_comp.x_data[:, 0]

bins = np.arange(lx_rat.min(), lx_rat.max(), 0.05)

plt.figure(figsize=(6, 6))
plt.hist(lx_rat, bins=bins, color='firebrick', alpha=0.7)
plt.minorticks_on()
plt.tick_params(which='both', direction='in', top=True, right=True)
plt.axvline(1, linestyle='dashed', color='black')
plt.xlabel(r'$L^{0.1-2.4\rm{keV}}_{\rm{XMM},500} / L^{0.1-2.4\rm{keV}}_{\rm{MCXC},500}$', fontsize=15)
plt.ylabel('N', fontsize=15)
plt.tight_layout()
plt.savefig('../../outputs/figures/global_properties/xmm_mcxc_lx_ratio.pdf')
plt.show()

### Comparing MCXC and X-LoVoCCS $R_{500}$ values

We have derived our own values of $R_{500}$ using the XGA-LTR, and measured temperatures and luminosities within them - we compare the $R_{500}$ values to those which were measured for the MCXC catalogue - again recall that we ran the pipeline twice, one with metallicity fixed and one with it free, so we will make comparisons with both:

In [ ]:
r500_comp = scaling_relation_curve_fit(straight_line, Quantity(results['r500'], 'kpc'), 
                                       Quantity(results['r500+-'], 'kpc'), 
                                       Quantity(results['MCXC_R500'], 'Mpc').to('kpc'), y_norm=Quantity(1, 'kpc'), 
                                       x_norm=Quantity(1, 'kpc'), point_names=results['LoVoCCS_name'].values, 
                                       y_name=r'$R^{\rm{XMM}}_{500}$', 
                                       x_name=r'$R^{\rm{MCXC}}_{500}$', 
                                       third_dim_info=results['redshift'].values, third_dim_name='Redshift')

r500_comp_mfree = scaling_relation_curve_fit(straight_line, Quantity(results_mfree['r500'], 'kpc'), 
                                       Quantity(results_mfree['r500+-'], 'kpc'), 
                                       Quantity(results_mfree['MCXC_R500'], 'Mpc').to('kpc'), y_norm=Quantity(1, 'kpc'), 
                                       x_norm=Quantity(1, 'kpc'), point_names=results_mfree['LoVoCCS_name'].values, 
                                       y_name=r'$R^{\rm{XMM}}_{500}$', 
                                       x_name=r'$R^{\rm{MCXC}}_{500}$', 
                                       third_dim_info=results_mfree['redshift'].values, third_dim_name='Redshift')

#### Metallicity free

In [ ]:
r500_comp_mfree.view(figsize=(8, 7), label_points=False, log_scale=False, x_lims=Quantity([400, 1300], 'kpc'), one_to_one=True, 
                     y_lims=Quantity([400, 1300], 'kpc'), 
                     save_path='../../outputs/figures/global_properties/xmm_mcxc_r500_mfree_comp.pdf')

print(r500_comp_mfree.par_names)
r500_comp_mfree.pars

In [ ]:
r500_comp_mfree.view(figsize=(8, 7), label_points=True, log_scale=False, x_lims=Quantity([400, 1300], 'kpc'), one_to_one=True, 
                     y_lims=Quantity([400, 1300], 'kpc'))
{ind: r500_comp_mfree.point_names[ind] for ind in range(0, len(r500_comp_mfree.point_names))}

We again calculate and plot the distribution of ratios, as another way of examining the same comparison:

In [ ]:
r500_rat = r500_comp_mfree.y_data[:, 0] / r500_comp_mfree.x_data[:, 0]

bins = np.arange(r500_rat.min(), r500_rat.max(), 0.05)

plt.figure(figsize=(6, 6))
plt.hist(r500_rat, bins=bins, color='tab:cyan', alpha=0.7)
plt.minorticks_on()
plt.tick_params(which='both', direction='in', top=True, right=True)
plt.axvline(1, linestyle='dashed', color='black')
plt.xlabel(r'$R^{\rm{XMM}}_{500} / R^{\rm{MCXC}}_{500}$', fontsize=15)
plt.ylabel('N', fontsize=15)

plt.tight_layout()
plt.savefig('../../outputs/figures/global_properties/xmm_mcxc_r500_mfree_ratio.pdf')
plt.show()

#### Metallicity fixed

In [ ]:
r500_comp.view(figsize=(8, 7), label_points=False, log_scale=False, x_lims=Quantity([400, 1300], 'kpc'), one_to_one=True, 
               y_lims=Quantity([400, 1300], 'kpc'), save_path='../../outputs/figures/global_properties/xmm_mcxc_r500_comp.pdf')

print(r500_comp.par_names)
r500_comp.pars

In [ ]:
r500_comp.view(figsize=(8, 7), label_points=True, log_scale=False, x_lims=Quantity([400, 1300], 'kpc'), one_to_one=True, 
               y_lims=Quantity([400, 1300], 'kpc'))
{ind: r500_comp.point_names[ind] for ind in range(0, len(r500_comp.point_names))}

We again calculate and plot the distribution of ratios, as another way of examining the same comparison:

In [ ]:
r500_rat = r500_comp.y_data[:, 0] / r500_comp.x_data[:, 0]

bins = np.arange(r500_rat.min(), r500_rat.max(), 0.05)

plt.figure(figsize=(6, 6))
plt.hist(r500_rat, bins=bins, color='tab:cyan', alpha=0.7)
plt.minorticks_on()
plt.tick_params(which='both', direction='in', top=True, right=True)
plt.axvline(1, linestyle='dashed', color='black')
plt.xlabel(r'$R^{\rm{XMM}}_{500} / R^{\rm{MCXC}}_{500}$', fontsize=15)
plt.ylabel('N', fontsize=15)

plt.tight_layout()
plt.savefig('../../outputs/figures/global_properties/xmm_mcxc_r500_ratio.pdf')
plt.show()